In [ ]:
with open('data/enwik8', 'rb') as f:
    data_all = f.read()

split = int(len(data_all) * 0.9)
data_train = data_all[:split]
data_test = data_all[split:]

In [ ]:
from ftm import prep

def train_iter(batch=4, seqlen=2048):
    data = [data_train]
    data = prep.frombytes(data)
    data = prep.chop(data, seqlen=seqlen*16)
    data = prep.shuffle(data, bufsize=128)
    data = prep.stack(data, batch=batch)
    data = prep.chop(data, seqlen=seqlen, dim=1)
    data = prep.shuffle(data, bufsize=128)
    return data

In [ ]:
import torch
from ftm.model import ModelArgs, Decformer

device = torch.device('cuda')
dtype = torch.bfloat16

decays = 0.5 ** (0.3 ** torch.arange(5))
config = ModelArgs(
    n_layers = 6,
    n_tokens = 256,
    n_embed = 1024,

    n_mem_dim = 2,
    n_mem_key = 256,
    n_mem_val = 256,

    n_mlp = 2048,
    decays = decays.tolist(),
    dropout = 0.1,
)

model = Decformer(config)

model.to(dtype).to(device)

try:
    model.load_state_dict(torch.load('./model.pt'))
except:
    pass

params = sum(p.numel() for p in model.parameters())
print(f'{params=:,}')